### N Gram Analysis According To Fourgram

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 1000  # 28 native word end index      

In [4]:
ngram_data_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Data/1-N Gram Data Prepare"

ngram_used_data_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Data/N Gram Used"


path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Result/2-N Gram Analysis According To Fourgram"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def strip(df):
    for i in df.columns:
        new_name = i.strip()
        df.rename(columns={f"{i}":f"{new_name}"}, inplace=True)
        df[f"{new_name}"] = df[f"{new_name}"].apply(lambda x: x.strip())
    
    return df

In [6]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [7]:
def word_in_wordgroup(source_word_list, df_target, target_column, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=False)]  # na=True failed  
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [8]:
def word_in_wordgroup_simple(df, source_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=False)]  # na=True failed    
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

#### Ngram Read

In [9]:
df_twogram = pd.read_csv(f"{ngram_data_path}/{lang_folder.capitalize()}_{word_end}_Word_Twogram_Selected.csv")
df_twogram

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
220163,dışarıda bizimle,6
220164,bizim kimseye,6
220165,tarafa bunu,6
220166,beri kahrolası,6


In [ ]:
df_twogram_used = pd.read_csv(f"{ngram_used_data_path}/{lang_folder.capitalize()}_Twogram_Used.csv")
df_twogram_used

In [10]:
df_threegram = pd.read_csv(f"{ngram_data_path}/{lang_folder.capitalize()}_{word_end}_Word_Threegram_Selected.csv")
df_threegram

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
500343,sonra insanlar bu,10
500344,o anda dünya,10
500345,zaman zaman sizinle,10
500346,bir güç sahibi,10


In [ ]:
df_threegram_used = pd.read_csv(f"{ngram_used_data_path}/{lang_folder.capitalize()}_Threegram_Selected.csv")
df_threegram_used

In [11]:
df_fourgram = pd.read_csv(f"{ngram_data_path}/{lang_folder.capitalize()}_{word_end}_Word_Fourgram_Selected.csv")
df_fourgram

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
366580,sadece yemek ve içki,7
366581,küçük bir şey ve,7
366582,olduğunu bu yüzden biliyorum,7
366583,sadece yeri değil zamanı,7


In [12]:
df_fourgram_selected_for_course = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment3/\
Data/Manuel Select/Selected 4grams.xlsx")
df_fourgram_selected_for_course

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
95,her şey iyi olacak,2479
96,öyle bir şey değil,2478
97,bunu sen mi yaptın,2453
98,ne demek oluyor bu,2431


#### Ngram In Ngram

In [13]:
twogram_list = df_twogram["twogram"].to_list()
twogram_used_list = df_twogram_used["twogram"].to_list()
threegram_list = df_threegram["threegram"].to_list()
threegram_used_list = df_threegram_used["threegram"].to_list()
fourgram_list = df_fourgram["fourgram"].to_list()
fourgram_course_list = df_fourgram_selected_for_course["fourgram"].to_list()

##### Twogram Used In Threegram Used (3A)

In [ ]:
# parameter
source_wordgroup_list = twogram_used_list
search_column_name = "twogram"
target_column = "threegram"
df_target = df_threegram_used

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

#search_list = list(searchlist)
#result_list = list(resultlist)
#df_word_in_two = pd.DataFrame([search_list,result_list]).transpose()
#df_word_in_two.columns = [f"{search_column_name}",f"{target_column}"]
#df_word_in_two

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_used_in_three_used = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_used_in_three_used

In [ ]:
df_two_used_in_threegram_used_list = df_two_used_in_three_used.groupby(["twogram"])["threegram"].apply(list).reset_index(name="threegram")
df_two_used_in_threegram_used_list

In [22]:
#df_two_used_in_threegram_used_list.to_excel(f"{lang_folder.capitalize()}_Twogram_Used_In_Threegram_Used_Result.xlsx", index=False)

##### Twogram Used In Selected Fourgram (2A)

In [ ]:
# parameter
source_wordgroup_list = threegram_used_list
search_column_name = "threegram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

#search_list = list(searchlist)
#result_list = list(resultlist)
#df_word_in_two = pd.DataFrame([search_list,result_list]).transpose()
#df_word_in_two.columns = [f"{search_column_name}",f"{target_column}"]
#df_word_in_two

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_used_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_used_in_selected_four

In [ ]:
df_two_used_in_selected_four_list = df_two_used_in_selected_four.groupby(["twogram"])["fourgram"].apply(list).reset_index(name="fourgram")
df_two_used_in_selected_four_list

In [ ]:
#df_two_used_in_selected_four_list.to_excel(f"{lang_folder.capitalize()}_Twogram_Used_In_Selected_Fourgram_Result.xlsx", index=False)

##### Threegram Used In Selected Fourgram (4A)

In [ ]:
# parameter
source_wordgroup_list = threegram_used_list
search_column_name = "threegram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

#search_list = list(searchlist)
#result_list = list(resultlist)
#df_word_in_two = pd.DataFrame([search_list,result_list]).transpose()
#df_word_in_two.columns = [f"{search_column_name}",f"{target_column}"]
#df_word_in_two

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_three_used_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_three_used_in_selected_four

In [ ]:
df_three_used_in_selected_four_list = df_three_used_in_selected_four.groupby(["threegram"])["fourgram"].apply(list).reset_index(name="fourgram")
df_three_used_in_selected_four_list

In [ ]:
#df_three_used_in_selected_four_list.to_excel(f"{lang_folder.capitalize()}_Threegram_Used_In_Selected_Fourgram_Result.xlsx", index=False)

##### Twogram Used In Threegram Used In Fourgram (1A)

In [ ]:
df_three_used_in_selected_four_list

In [ ]:
# parameter
source_wordgroup_list = twogram_used_list
search_column_name = "twogram"
target_column = "threegram"
df_target = df_three_used_in_selected_four_list

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

#search_list = list(searchlist)
#result_list = list(resultlist)
#df_word_in_two = pd.DataFrame([search_list,result_list]).transpose()
#df_word_in_two.columns = [f"{search_column_name}",f"{target_column}"]
#df_word_in_two

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_twogram_used_in_three_used = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_twogram_used_in_three_used

In [ ]:
df_twogram_used_in_three_used_in_selected_four = pd.merge(df_twogram_used_in_three_used, df_three_used_in_selected_four_list, how="inner", on="threegram")
df_twogram_used_in_three_used_in_selected_four

In [ ]:
#df_three_used_in_four_list = df_three_used_in_four.groupby(["twogram"])["threegram"].apply(list).reset_index(name="threegram")
#df_three_used_in_four_list

In [ ]:
#df_twogram_used_in_three_used_in_selected_four.to_excel(f"{lang_folder.capitalize()}_Twogram_Used_In_Threegram_Used_In_Selected_Fourgram_Result.xlsx", index=False)

##### Twogram In Threegram (3B)

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "threegram"
df_target = df_threegram

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

#search_list = list(searchlist)
#result_list = list(resultlist)
#df_word_in_two = pd.DataFrame([search_list,result_list]).transpose()
#df_word_in_two.columns = [f"{search_column_name}",f"{target_column}"]
#df_word_in_two

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_three = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_in_three

In [ ]:
df_two_in_threegram_list = df_two_in_three.groupby(["twogram"])["threegram"].apply(list).reset_index(name="threegram")
df_two_in_threegram_list

In [ ]:
#df_two_in_threegram_list.to_excel(f"{lang_folder.capitalize()}_Twogram_In_Threegram_Result.xlsx", index=False)

##### Twogram In Selected Fourgram (2B)

In [14]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

#search_list = list(searchlist)
#result_list = list(resultlist)
#df_word_in_two = pd.DataFrame([search_list,result_list]).transpose()
#df_word_in_two.columns = [f"{search_column_name}",f"{target_column}"]
#df_word_in_two

In [15]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_in_selected_four

,twogram,fourgram
0,bir şey,bir şey var mı
1,bir şey,gereken bir şey var
2,bir şey,başka bir şey var
3,bir şey,bir şey daha var
4,bir şey,bir şey mi var
...,...,...
295,mi hayır,evet mi hayır mı
296,yapıyorsun değil,şaka yapıyorsun değil mi
297,şey buldun,bir şey buldun mu
298,öyle ya,öyle ya da böyle


In [16]:
df_two_in_selected_four_list = df_two_in_selected_four.groupby(["twogram"])["fourgram"].apply(list).reset_index(name="fourgram")
df_two_in_selected_four_list

,twogram,fourgram
0,bana yardım,[bana yardım eder misin]
1,başka bir,"[başka bir şey var, başka bir şey yok, başka b..."
2,ben de,"[ben de seni seviyorum, evet ben de öyle]"
3,benim için,[benim için çok önemli]
4,biliyor musun,"[ne var biliyor musun, ne olduğunu biliyor mus..."
...,...,...
185,şey yolunda,[her şey yolunda mı]
186,şeye ihtiyacın,[bir şeye ihtiyacın olursa]
187,şeyi yaparım,[için her şeyi yaparım]
188,şeyin var,[bir şeyin var mı]


In [17]:
#df_two_in_selected_four_list.to_excel(f"{lang_folder.capitalize()}_Twogram_In_Selected_Fourgram_Result.xlsx", index=False)

##### Threegram In Selected Fourgram (4B)

In [18]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

#search_list = list(searchlist)
#result_list = list(resultlist)
#df_word_in_two = pd.DataFrame([search_list,result_list]).transpose()
#df_word_in_two.columns = [f"{search_column_name}",f"{target_column}"]
#df_word_in_two

In [19]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_three_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_three_in_selected_four

,threegram,fourgram
0,bir şey yok,diye bir şey yok
1,bir şey yok,başka bir şey yok
2,bir şey yok,bir şey yok mu
3,bir şey yok,hiç bir şey yok
4,bir şey yok,öyle bir şey yok
...,...,...
195,sakin ol tamam,sakin ol tamam mı
196,istiyor musun istemiyor,istiyor musun istemiyor musun
197,şey söyleyeyim mi,bir şey söyleyeyim mi
198,olur ne olmaz,ne olur ne olmaz


In [20]:
df_three_in_selected_four_list = df_three_in_selected_four.groupby(["threegram"])["fourgram"].apply(list).reset_index(name="fourgram")
df_three_in_selected_four_list

,threegram,fourgram
0,bana yardım eder,[bana yardım eder misin]
1,başka bir şey,"[başka bir şey var, başka bir şey yok, başka b..."
2,ben de seni,[ben de seni seviyorum]
3,ben de öyle,[evet ben de öyle]
4,benim için çok,[benim için çok önemli]
...,...,...
158,şey yok mu,[bir şey yok mu]
159,şey yolunda mı,[her şey yolunda mı]
160,şeye ihtiyacın olursa,[bir şeye ihtiyacın olursa]
161,şeyin var mı,[bir şeyin var mı]


In [21]:
#df_three_in_selected_four_list.to_excel(f"{lang_folder.capitalize()}_Threegram_In_Selected_Fourgram_Result.xlsx", index=False)

##### Twogram In Threegram In Selected Fourgram (1B)

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "threegram"
df_target = df_three_in_selected_four_list

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

#search_list = list(searchlist)
#result_list = list(resultlist)
#df_word_in_two = pd.DataFrame([search_list,result_list]).transpose()
#df_word_in_two.columns = [f"{search_column_name}",f"{target_column}"]
#df_word_in_two

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_three = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_in_three

,twogram,fourgram
0,bir şey,bir şey var mı
1,bir şey,gereken bir şey var
2,bir şey,başka bir şey var
3,bir şey,bir şey daha var
4,bir şey,bir şey mi var
...,...,...
295,mi hayır,evet mi hayır mı
296,yapıyorsun değil,şaka yapıyorsun değil mi
297,şey buldun,bir şey buldun mu
298,öyle ya,öyle ya da böyle


In [ ]:
df_twogram_in_three_in_selected_four = pd.merge(df_two_in_three, df_three_in_selected_four_list, how="inner", on="threegram")
df_twogram_in_three_in_selected_four

,twogram,fourgram
0,bana yardım,[bana yardım eder misin]
1,başka bir,"[başka bir şey var, başka bir şey yok, başka b..."
2,ben de,"[ben de seni seviyorum, evet ben de öyle]"
3,benim için,[benim için çok önemli]
4,biliyor musun,"[ne var biliyor musun, ne olduğunu biliyor mus..."
...,...,...
185,şey yolunda,[her şey yolunda mı]
186,şeye ihtiyacın,[bir şeye ihtiyacın olursa]
187,şeyi yaparım,[için her şeyi yaparım]
188,şeyin var,[bir şeyin var mı]


In [ ]:
#df_twogram_in_three_in_selected_four.to_excel(f"{lang_folder.capitalize()}_Twogram_In_Threegram_In_Selected_Fourgram_Result.xlsx", index=False)

#### Copy Move And Delete

In [114]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_Text_Selection_Result*")
#output_file = glob.glob(f"*_Selected_*.xlsx")
output_file

['Turkish_Word_Text_Selection_Result.csv',
 'Turkish_Ninegram_Text_Selection_Result.csv',
 'Turkish_Eightgram_Text_Selection_Result.csv',
 'Turkish_Sevengram_Text_Selection_Result.csv']

In [115]:
for l in output_file:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [116]:
for j in output_file:
    try:
        os.remove(j)
    except:
        pass